In [1]:
from kafka import KafkaConsumer
from kafka import TopicPartition
from datetime import datetime
import pandas as pd
from cassandra.cluster import Cluster
import json
from botocore.vendored import requests
import findspark
findspark.init()
import pyspark.sql 
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
import pandas as pd
from cassandra.cluster import Cluster
import numpy as np
from pyspark.sql.types import *

In [2]:
# #------------------------------- SPARK CONTEXT -------------------------------
# sc = SparkContext('local')
# spark = SparkSession(sc)
# spark = SparkSession.builder.appName('CC_Fraud').getOrCreate()


In [3]:
#------------------------------- Load Model from File System -------------------------------

def MachineLearningModelLoading():
	path="./model"
	newmodel=PipelineModel.load(path)
	return newmodel
	
newmodel = MachineLearningModelLoading()


Py4JJavaError: An error occurred while calling o24.partitions.
: java.lang.UnsatisfiedLinkError: org.apache.hadoop.io.nativeio.NativeIO$POSIX.stat(Ljava/lang/String;)Lorg/apache/hadoop/io/nativeio/NativeIO$POSIX$Stat;
	at org.apache.hadoop.io.nativeio.NativeIO$POSIX.stat(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$POSIX.getStat(NativeIO.java:608)
	at org.apache.hadoop.fs.RawLocalFileSystem$DeprecatedRawLocalFileStatus.loadPermissionInfoByNativeIO(RawLocalFileSystem.java:934)
	at org.apache.hadoop.fs.RawLocalFileSystem$DeprecatedRawLocalFileStatus.loadPermissionInfo(RawLocalFileSystem.java:848)
	at org.apache.hadoop.fs.RawLocalFileSystem$DeprecatedRawLocalFileStatus.getPermission(RawLocalFileSystem.java:816)
	at org.apache.hadoop.fs.LocatedFileStatus.<init>(LocatedFileStatus.java:52)
	at org.apache.hadoop.fs.FileSystem$4.next(FileSystem.java:2199)
	at org.apache.hadoop.fs.FileSystem$4.next(FileSystem.java:2179)
	at org.apache.hadoop.mapred.FileInputFormat.singleTh

In [1]:
pip install pyspark

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\Users\saritajoshi\Documents\Repos\Credit-Card-Fraud-Detection-Spark\venv\Scripts\python.exe -m pip <command> [options]

no such option: -I


In [ ]:


# myschema = StructType([ StructField("txnid",StringType(), True)\
#                      ,StructField("age", IntegerType(), True)\
#                      ,StructField("amount", FloatType(), True)\
#                      ,StructField("cc_no", LongType(), True)\
#                      ,StructField("cc_provider", IntegerType(), True)\
#                      ,StructField("country_code", StringType(), True)\
#                      ,StructField("country_index", IntegerType(), True)\
#                      ,StructField("country_name", StringType(), True)\
#                      ,StructField("day", IntegerType(), True)\
#                      ,StructField("gender", StringType(), True)\
#                      ,StructField("gender_index", IntegerType(), True)\
#                      ,StructField("hour", IntegerType(), True)\
#                      ,StructField("loan", StringType(), True)\
#                      ,StructField("loan_index", IntegerType(), True)\
#                      ,StructField("marital_status", StringType(), True)\
#                      ,StructField("marital_status_index", IntegerType(), True)\
#                      ,StructField("merchant", StringType(), True)\
#                      ,StructField("merchant_index", IntegerType(), True)\
#                      ,StructField("minute", IntegerType(), True)\
#                      ,StructField("month", IntegerType(), True)\
#                      ,StructField("status", StringType(), True)\
#                      ,StructField("status_index", IntegerType(), True)\
#                      ,StructField("txn_time", StringType(), True)\
#                      ,StructField("year", IntegerType(), True)\
#                     ])


In [ ]:
#------------------------------------ Data Cleaning -------------------------------------

def predicting_status(data_all,newmodel):
	#print("Number of rows: {},  Number of coulumns : {}".format(data_all.count(), len(data_all.columns)))

	#Replace "-" with "_" in column names
	columns_new = [col.replace("-", "_") for col in data_all.columns]
	data_all = data_all.toDF(*columns_new)

	#Print Schema and show top 5 row
	#data_all.printSchema()

	#label = "status_index"
	xvars = ['trans_date_trans_time', 'cc_num', 'merchant', 'category', 'amt', 'first', 'last', 'gender', 'street', 'city', 'state', 'zip', 'lat', 'long', 'city_pop', 'job', 'dob', 'trans_num', 'unix_time', 'merch_lat', 'merch_long', 'is_fraud'] #all numeric
	#print("label = {}".format(label))
	#print("features = {}".format(xvars))
	select_cols = xvars
	#select_cols.append(label)
	data = data_all.select(select_cols)
	pred1 = newmodel.transform(data)
    
	return pred1.select("prediction")

In [ ]:

# #------------------------------------------- CASSANDRA CONNECTION -------------------------------------------------------
# cluster = Cluster()
# session = cluster.connect()
# session.execute("CREATE KEYSPACE IF NOT EXISTS bigdata WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 3}")
# session.set_keyspace("bigdata")
# #session.execute("DROP TABLE testing")
# session.execute("CREATE TABLE IF NOT EXISTS testing (txn_id varchar PRIMARY KEY,cc_no bigint ,txn_time varchar, year smallint, month smallint, day smallint, hour smallint, minute smallint,amount double, cc_provider int, merchant varchar,merchant_index smallint, country_name varchar, country_code varchar,country_index smallint, age int, marital_status varchar,marital_status_index smallint,gender varchar,gender_index smallint,loan varchar,loan_index smallint, status varchar,status_index smallint)")
# # session.set_keyspace("mykeyspace")
# # session.execute("CREATE TABLE IF NOT EXISTS training (txn_id varchar PRIMARY KEY,cc_no bigint , cc_provider int, year smallint, month smallint, day smallint, hour smallint, min smallint,txn_time varchar, merchant varchar, location varchar, country varchar, amount float, status varchar)")


In [27]:
topic_name = 'transaction_data'
consumer = KafkaConsumer(topic_name, group_id='my-first-application', bootstrap_servers=['localhost:9092'])
dd= {}
for msg in consumer:
	
	msg=msg.value.decode("utf-8")
	df = spark.read.json(sc.parallelize([msg]))
	msg=json.loads(msg)
	dd = msg
	print(msg)
	break
	status_predicted=predicting_status(df,newmodel)
	if status_predicted.collect()[0][0]==0:
		status="Approved"
	else:
		status="Rejected"

	msg["status"]=status
	elastic_data={
	"cc_no":msg["cc_no"],
	"txn_time":msg["txn_time"],
	"year":msg["year"],
	"month":msg["month"],
	"cc_provider":msg["cc_provider"],
	"merchant":msg["merchant"],
	"country_code":msg["country_code"],
	"amount":msg["amount"],
	"age":msg["age"],
	"marital_status":msg["marital_status"],
	"gender":msg["gender"],
	"loan":msg["loan"],
	"status":msg["status"]
	}
	
	# strq=" INSERT INTO testing JSON "+"'"+json.dumps(msg)+"'"
	# session.execute(strq)


In [ ]:
dd

{'Unnamed: 0': 0,
 'trans_date_trans_time': '2020-06-21 12:14:25',
 'cc_num': 2291163933867244,
 'merchant': 'fraud_Kirlin and Sons',
 'category': 'personal_care',
 'amt': 2.86,
 'first': 'Jeff',
 'last': 'Elliott',
 'gender': 'M',
 'street': '351 Darlene Green',
 'city': 'Columbia',
 'state': 'SC',
 'zip': 29209,
 'lat': 33.9659,
 'long': -80.9355,
 'city_pop': 333497,
 'job': 'Mechanical engineer',
 'dob': '1968-03-19',
 'trans_num': '2da90c7d74bd46a0caf3777415b3ebd3',
 'unix_time': 1371816865,
 'merch_lat': 33.986391,
 'merch_long': -81.200714,
 'is_fraud': 0}

In [23]:
	
consumer.close()

In [7]:
spark.stop()